In [806]:
import numpy as np
from scipy.linalg import logm, expm

from ncpol2sdpa import generate_variables, SdpRelaxation

from qutip import sigmax, sigmay, sigmaz, propagator, tensor, Qobj

from sympy import Matrix, eye, diag, Float, Rational, symbols, I, re
from sympy.matrices import randMatrix, matrix2numpy
from sympy.physics.quantum import TensorProduct

In [14]:
sx = Matrix(sigmax())

In [463]:
expm(1j * np.array(sigmax()))

array([[0.54030231+0.j        , 0.        +0.84147098j],
       [0.        +0.84147098j, 0.54030231+0.j        ]])

In [17]:
a = tensorproduct(sx, eye(2))

In [19]:
a.is_Matrix

False

The function generating 
$N\Delta t \left[\hat{1} \otimes \hat{H}_0 + u((\hat{T} + \hat{1}/2) \Delta t ) \otimes \hat{V}\right]$

In [434]:
def control_gen(*, H0, V, N, dt, u, **kwargs):
    
    # Evaluate u((\hat{T} + \hat{1}/2) \Delta t )
    u_term = diag(*(
        u(dt * (k +  Rational(1, 2))) for k in range(N)
    ))
        
    return N * dt * (TensorProduct(eye(N), H0) + TensorProduct(u_term, V))

The cost function to be minimized 
$\left\| N\Delta t \left[\hat{1} \otimes \hat{H}_0 + u((\hat{T} + \hat{1}/2) \Delta t ) \otimes \hat{V}\right] + \hat{1} \otimes \hat{G}\right\|_F^2 \to \min.$

In [484]:
def J(*, N, G, **kwargs):
    return (
        control_gen(N=N, **kwargs) + TensorProduct(eye(N), G)
    ).norm() ** 2

In [485]:
def MatrixJ(*, N, G, **kwargs):
    return        control_gen(N=N, **kwargs) + TensorProduct(eye(N), G)
    

In [486]:
def expm_control_gen(*, N, G, **kwargs):
    return (-1j * control_gen(N=N, **kwargs)).exp().evalf()

In [490]:
def expG(*, N, G, **kwargs):
    return TensorProduct(eye(N), (1j * G).exp()).evalf()

# Example

In [838]:
H0 = randMatrix(4, percent=100) / 100 + I * randMatrix(4, percent=80) / 100
H0 = H0.H + H0

V = randMatrix(4, percent=100) / 100 + I * randMatrix(4, percent=80) / 100
V = V.H + V

sys_params = dict(
    H0 = H0,
    V = V,
        
    dt = Rational(1, 10),
    N = 10,
)

H0 = Qobj(matrix2numpy(sys_params['H0']))
V = Qobj(matrix2numpy(sys_params['V']))

In [840]:
U = propagator([H0, [V, 't ** 2']], 4)

In [843]:
sys_params['G'] = Matrix(np.around(logm(U), 4) / 1j)

In [844]:
sys_params['G']

Matrix([
[           0.7528, -0.3401 - 0.4321*I,  -0.428 - 1.0452*I,  0.0113 + 0.3432*I],
[-0.3401 + 0.432*I,            -0.6011, -0.7467 + 0.2925*I, -1.2395 + 0.5125*I],
[-0.428 + 1.0451*I, -0.7467 - 0.2924*I,            -0.0009,  -0.8962 + 1.272*I],
[0.0113 - 0.3432*I, -1.2395 - 0.5125*I, -0.8963 - 1.2721*I,            -0.0326]])

In [845]:
(obj)

266.682*x0**2 + 266.682*x0*x1 + 177.34353*x0*x2 + 395.507844*x0 + 88.671765*x1**2 + 132.674295*x1*x2 + 197.753922*x1 - 4.99600361081321e-16*I*x1 + 52.8927078225*x2**2 + 131.50635813*x2 - 4.99600361081321e-16*I*x2 + 240.323434188

In [846]:
x = generate_variables('x', 3)

obj = J(u=lambda t: x[0] + x[1] * t + x[2] * t ** 2, **sys_params).simplify()

sdp = SdpRelaxation(x)
sdp.get_relaxation(8, objective=re(obj))
sdp.solve(solver='mosek')

# extract the values of control
if sdp.status == 'optimal':
    print([sdp[_] for _ in sdp.variables])

[-0.6482934195176792, 3.693863442391718e-08, 2.6555582661506082e-08]


In [847]:
Un = propagator(
    [H0, 
    #[V, '1.2106486632912106 * t -1.0164572106234302 * t * t']],
     [V, '-0.6482934195176792']],
    4
)


In [848]:
(Un - U).norm()

5.395916941146283

In [822]:
U.norm()

3.9999999999998366

In [742]:
Un.norm()

3.9999999999999956

In [765]:
sys_params['G']

Matrix([
[             -0.2114,  -0.2135 - 0.08872*I, -0.58334 - 0.20692*I, -0.28785 - 0.14271*I],
[ -0.2135 + 0.08872*I,             -0.38259,    0.0483 + 0.1738*I, -0.81116 - 0.00155*I],
[-0.58334 + 0.20692*I,   0.0483 - 0.17379*I,             -0.81325, -0.65112 + 0.02163*I],
[-0.28785 + 0.14272*I, -0.81116 + 0.00155*I, -0.65112 - 0.02163*I,             -1.02695]])

In [767]:
Matrix(np.around(logm(Un), 5) / 1j)

Matrix([
[-0.04192,    -0.22, -0.52251,  -0.1767],
[   -0.22, -0.39275,        0, -0.87575],
[-0.52251,        0, -0.96071, -0.70856],
[ -0.1767, -0.87575, -0.70856, -1.05964]])

In [723]:
Un

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.82344125-0.13371235j -0.09627055-0.06197975j -0.29435129-0.40008505j
  -0.20137559+0.06077131j]
 [-0.09627055-0.06197975j  0.54415338-0.74110819j -0.19945144+0.05067774j
  -0.25107518-0.19015358j]
 [-0.29435128-0.40008504j -0.19945144+0.05067774j  0.32640483-0.51100513j
  -0.43763869-0.38954611j]
 [-0.20137559+0.06077131j -0.25107518-0.19015358j -0.43763869-0.38954611j
   0.62011482-0.35880552j]]